In [1]:
import optuna as opt
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

import sys
sys.path.append('/home/ubuntu/nk-paper-2024/pscapes')
sys.path.append('/home/ubuntu/nk-paper-2024/nk-ml-2024')

from torch.utils.data import DataLoader

from pscapes.landscape_class import ProteinLandscape
from pscapes.utils import dict_to_np_array, np_array_to_dict

from src.architectures.architectures import SequenceRegressionCNN
from src.architectures.ml_utils import train_val_test_split_ohe


/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEQ_LEN = 6
AA_ALPHABET = 'ACDEFG'

In [3]:
#Load NK landscapes -- only a single replicate for hparam tuning 

LANDSCAPES = []
for k in range(6): 
    for r in range(1): 
        landscape = ProteinLandscape(csv_path='../data/nk_landscapes/k{0}_r{1}.csv'.format(k,r), amino_acids=AA_ALPHABET)
        LANDSCAPES.append(landscape)

In [4]:
LANDSCAPES = [i.fit_OHE() for i in LANDSCAPES]

In [5]:
landscapes_ohe, xy_train, xy_val, xy_test, x_test, y_test = train_val_test_split_ohe(LANDSCAPES)

In [6]:
len(xy_train)

6

In [9]:
torch.cuda.is_available()

True

In [7]:
landscape0_xy_train = xy_train[0]
landscape0_xy_val   = xy_val[0]

In [33]:
class EarlyStoppingHyperOpt:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0

    def should_stop(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1

        return self.counter >= self.patience

In [50]:
def cnn_objective_NK(trial, train_data, val_data, epochs):
    # Define the search space
    num_conv_layers = trial.suggest_int('num_conv_layers', 1, 2)
    
    num_kernels = [int(trial.suggest_discrete_uniform("n_kernels", 16, 128, 16))
                   for i in range(num_conv_layers)]  
    
    kernel_sizes = [int(trial.suggest_discrete_uniform("kernel_sizes", 2, 6, 1))
                   for i in range(num_conv_layers)]
    
    learning_rate = trial.suggest_categorical('lr', [0.01, 0.001, 0.0001])

    batch_size    = trial.suggest_discrete_uniform('batch_size', 32, 128, 32)
    
    # Initialize model with the trial’s hyperparameters
    model = SequenceRegressionCNN(input_channels=len(AA_ALPHABET), sequence_length=SEQ_LEN, 
                                  num_conv_layers=num_conv_layers, n_kernels=num_kernels, kernel_sizes=kernel_sizes)
    
    # Loss and optimizer
    loss_fn = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Dummy training and validation data loaders
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32)


    # Early stopping setup
    early_stopping = EarlyStopping(patience=8, min_delta=1e-5)
    # Training loop with validation loss calculation
    for epoch in range(epochs):


        
        model.to(device)
        model.train()
        for x_batch, y_batch in train_loader:

            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            predictions = model(x_batch)
            loss = loss_fn(predictions, y_batch)
            loss.backward()
            optimizer.step()

        # Calculate validation loss`
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                predictions = model(x_batch)
                val_loss += loss_fn(predictions, y_batch).item()

        val_loss /= len(val_loader)
        
        
        trial.report(val_loss, epoch)

        print('Epoch: {0}, Val loss: {1}'.format(epoch, val_loss))
        # Check early stopping
        if early_stopping.should_stop(val_loss):
            print(f"Early stopping at epoch {epoch}")
            break
        
        
        if trial.should_prune():
            raise opt.TrialPruned()
    print('Best Val Loss this Trial: {}'.format(val_loss))
        

    return val_loss



In [51]:
import time

In [52]:
t1 = time.time()

In [53]:
t2 = time.time()

In [54]:
t2-t1

0.1602191925048828

In [55]:
# Running the study
torch.backends.nnpack.enabled = False

# creating studies
cnn_studies = [opt.create_study(direction='minimize') for i in LANDSCAPES]

#Loop for hparam opt

times = [time.time()] 

for index, study in enumerate(cnn_studies):
    study.optimize(lambda trial: cnn_objective(trial, train_data= xy_train[index], val_data=xy_val[index],
                                               epochs=50), n_trials=10)
    t = time.time()
    times.append(t)


[I 2024-10-28 05:53:06,284] A new study created in memory with name: no-name-38e5ed66-9187-4cf7-b443-84698f61f10f
[I 2024-10-28 05:53:06,285] A new study created in memory with name: no-name-3f120a35-131f-4bc5-8392-18e87946cce0
[I 2024-10-28 05:53:06,286] A new study created in memory with name: no-name-e4d473b3-bded-4fb1-80d8-022c0b3474bb
[I 2024-10-28 05:53:06,287] A new study created in memory with name: no-name-c7db90b0-a272-498c-913d-f4f8996fef76
[I 2024-10-28 05:53:06,287] A new study created in memory with name: no-name-086ab3ef-5604-4638-952e-3ef581cf6852
[I 2024-10-28 05:53:06,288] A new study created in memory with name: no-name-4ee9b115-da5b-43f5-99f8-abfccf66b0d9
/tmp/ipykernel_3136/3870252311.py:5: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  num_kernels = [int(trial.suggest_discrete_uniform("n_kernels", 16,

Epoch: 0, Val loss: 9.287287989575575e-05
Epoch: 1, Val loss: 2.3526344788339113e-05
Epoch: 2, Val loss: 2.2399774924739395e-05
Epoch: 3, Val loss: 7.48212672226503e-05
Epoch: 4, Val loss: 0.00024522188360629697
Epoch: 5, Val loss: 0.00018537779051723937


[I 2024-10-28 05:53:14,590] Trial 0 finished with value: 0.00011691222871921392 and parameters: {'num_conv_layers': 1, 'n_kernels': 80.0, 'kernel_sizes': 3.0, 'lr': 0.0040048549245837945}. Best is trial 0 with value: 0.00011691222871921392.


Epoch: 6, Val loss: 0.00011691222871921392
Early stopping at epoch 6
Best Val Loss this Trial: 0.00011691222871921392
Epoch: 0, Val loss: 0.00028452477116897924
Epoch: 1, Val loss: 0.00012260048230313975
Epoch: 2, Val loss: 9.490307506622098e-05
Epoch: 3, Val loss: 6.269825911674247e-05
Epoch: 4, Val loss: 3.3928447332710544e-05
Epoch: 5, Val loss: 5.3067704434825e-05
Epoch: 6, Val loss: 2.3565690341464277e-05
Epoch: 7, Val loss: 1.5033357766518218e-05
Epoch: 8, Val loss: 2.223719503746953e-05
Epoch: 9, Val loss: 1.7903752504779786e-05
Epoch: 10, Val loss: 1.0326172697072889e-05
Epoch: 11, Val loss: 9.660997442101028e-06
Epoch: 12, Val loss: 1.6343922710896153e-05
Epoch: 13, Val loss: 6.7320581625904005e-06
Epoch: 14, Val loss: 6.808584585942454e-06


[I 2024-10-28 05:53:33,056] Trial 1 finished with value: 8.919544895364266e-06 and parameters: {'num_conv_layers': 1, 'n_kernels': 48.0, 'kernel_sizes': 6.0, 'lr': 0.0010844762145890547}. Best is trial 1 with value: 8.919544895364266e-06.


...

Epoch: 0, Val loss: 0.019985601568642337
Epoch: 0, Val loss: 0.019741708233665962
Epoch: 1, Val loss: 0.019697947622650962


[I 2024-10-28 06:17:11,514] Trial 8 pruned. 


Epoch: 2, Val loss: 0.02124855349349797


[I 2024-10-28 06:17:12,695] Trial 9 pruned. 


Epoch: 0, Val loss: 0.02266710712853023


In [59]:
(times[-1]-times[0])/60

24.106786930561064

In [66]:
cnn_studies[-1].best_params

{'num_conv_layers': 2,
 'n_kernels': 80.0,
 'kernel_sizes': 4.0,
 'lr': 0.00031280142241287256}

In [63]:
cnn_studies[-1].best_value

0.019444677751893416

In [64]:
best_params = [i.best_params for i in cnn_studies]
best_scores = [i.best_value for i in cnn_studies]

In [67]:
import pickle

a = {'hello': 'world'}

with open('testCNN_optuna.pkl', 'wb') as handle:
    pickle.dump(cnn_studies, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [84]:
with open('hyperopt_CNN_NK_studies.pkl', 'rb') as handle:
    b = pickle.load(handle)



In [88]:
best_params = [i.best_params for i in b]
best_score  = [i.best_value for i in b]

In [89]:
best_params

[{'num_conv_layers': 2,
  'n_kernels_layer0': 128,
  'n_kernels_layer1': 64,
  'kernel_size_layer0': 5,
  'kernel_size_layer1': 5,
  'lr': 0.01,
  'batch_size': 32},
 {'num_conv_layers': 2,
  'n_kernels_layer0': 96,
  'n_kernels_layer1': 96,
  'kernel_size_layer0': 3,
  'kernel_size_layer1': 5,
  'lr': 0.001,
  'batch_size': 32},
 {'num_conv_layers': 2,
  'n_kernels_layer0': 96,
  'n_kernels_layer1': 96,
  'kernel_size_layer0': 5,
  'kernel_size_layer1': 3,
  'lr': 0.001,
  'batch_size': 64},
 {'num_conv_layers': 2,
  'n_kernels_layer0': 128,
  'n_kernels_layer1': 96,
  'kernel_size_layer0': 5,
  'kernel_size_layer1': 5,
  'lr': 0.001,
  'batch_size': 64},
 {'num_conv_layers': 2,
  'n_kernels_layer0': 96,
  'n_kernels_layer1': 128,
  'kernel_size_layer0': 3,
  'kernel_size_layer1': 5,
  'lr': 0.0001,
  'batch_size': 64},
 {'num_conv_layers': 2,
  'n_kernels_layer0': 96,
  'n_kernels_layer1': 32,
  'kernel_size_layer0': 3,
  'kernel_size_layer1': 5,
  'lr': 0.01,
  'batch_size': 32}]